In [59]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
d = {}

l1, l2, l3, l4 = [], [], [], []
ls = []
with open('result4.txt', 'r') as f:
    lines = f.readlines()
for line in lines:
    line = line.strip().split()
    line[2] = line[2].split(',')

    ls.append(line[0])
    for i in range(len(line[2])):
        l1.append(line[0])
        l2.append(line[1])
        l3.append(i + 1)
        l4.append(line[2][i])
da = pd.DataFrame({'OrderNo': l4, 'GroupNo':l2, 'WorkerNo':l3, 'TaskNo':l4})
da.head(10)


,OrderNo,GroupNo,WorkerNo,TaskNo
0,D0674,1,1,D0674
1,D0601,2,1,D0601
2,D0613,2,2,D0613
3,D0912,3,1,D0912
4,D0747,3,2,D0747
5,D0726,4,1,D0726
6,D0630,4,2,D0630
7,D0352,5,1,D0352
8,D0630,1,1,D0630
9,D0747,1,2,D0747


In [34]:
da['GroupNo'] = da['GroupNo'].astype(int)
da['WorkerNo'] = da['WorkerNo'].astype(int)
da = da.sort_values(by=['OrderNo','GroupNo','WorkerNo'],ascending=True)
da

,OrderNo,GroupNo,WorkerNo,TaskNo
843,D0026,1,5,D0026
873,D0026,3,1,D0026
845,D0096,1,7,D0096
687,D0096,2,3,D0096
713,D0096,3,1,D0096
...,...,...,...,...
336,D0912,5,3,D0912
467,D0912,5,3,D0912
502,D0912,5,5,D0912
604,D0912,5,5,D0912


In [ ]:
da.to_csv('result3.csv',index=None)


In [40]:
ls = []
with open('result59.csv','r') as f:
    k = f.readlines()
for line in k:
    ls.extend(line.strip().split(','))
print(ls)

['D0912', 'D0601', 'D0630', 'D0352', 'D0613', 'D0747', 'D0674', 'D0726', 'D0497', 'D0776', 'D0337', 'D0903', 'D0423', 'D0581', 'D0294', 'D0391', 'D0539', 'D0384', 'D0496', 'D0312', 'D0246', 'D0096', 'D0148', 'D0614', 'D0126', 'D0026', 'D0212', 'D0243', 'D0882', 'D0771', 'D0486', 'D0413', 'D0320', 'D0498', 'D0670', 'D0474', 'D0401', 'D0572', 'D0198', 'D0505', 'D0591', 'D0825', 'D0612', 'D0433', 'D0540', 'D0550', 'D0468', 'D0306', 'D0409', 'D0399', 'D0190', 'D0318', 'D0110', 'D0092', 'D0091', 'D0459', 'D0480', 'D0680', 'D0482', 'D0846', 'D0551', 'D0562', 'D0679', 'D0880', 'D0690', 'D0728', 'D0841', 'D0541', 'D0900', 'D0435', 'D0893', 'D0432', 'D0477', 'D0565', 'D0470', 'D0429', 'D0647', 'D0031', 'D0188', 'D0003', 'D0583', 'D0115', 'D0180', 'D0809', 'D0204', 'D0134', 'D0114', 'D0175', 'D0002', 'D0164', 'D0225', 'D0149', 'D0145', 'D0800', 'D0494', 'D0852', 'D0315', 'D0781', 'D0430', 'D0223', 'D0406', 'D0113', 'D0628', 'D0525', 'D0464', 'D0080', 'D0329', 'D0865', 'D0029', 'D0245', 'D0205', 

In [62]:
# -*- coding: UTF-8 -*-
import pandas as pd
import numpy as np
from multiprocessing import Pool
import random

import warnings


def initpara():
    alpha = 0.99
    t = (1, 100)
    markovlen = 1000
    return alpha, t, markovlen

warnings.filterwarnings('ignore')
with open('result59.csv', 'r') as f:
    lines = f.readlines()
indexs = 49

data = []

for line in lines:
    line = line.strip("'\n ").split(',')
    data.append(line)
with open('result.txt', 'r') as f:
    lines = f.readlines()
d = {}
for line in lines:
    line = line.strip()
    d[line.split()[0]] = line.split()[1].split(',')

matrix = pd.read_csv('met.csv', index_col=0)

matrix = matrix.loc[data[indexs]]
matrix = matrix[d.get(str(indexs))]

matrix = matrix.T

right = matrix.apply(lambda k: len(list(k)) - list(k)[::-1].index(1)).tolist()
left = matrix.apply(lambda k: list(k).index(1) + 1).tolist()
pos = [[left[j], right[j], data[indexs][j]] for j in range(len(left))]

# 模拟退火算法
# =================初始化参数============
num = len(pos)

solutionnew = []
solutioncurrent = solutionnew.copy()
valuecurrent = 990000

solutionbest = solutionnew.copy()
valuebest = 990000  # np.max

alpha, t2, markovlen = initpara()
t = t2[1]

result = []  # 记录迭代过程当中的最优解
while t > 1:
    for i in range(markovlen):
        # 订单起止区间信息
        ls = pos
        # 记录员工位置信息
        order = [[], [], [], [], []]

        # np.random.rand()产生[0, 1)区间的均匀随机数
        while True:  # 产生两个不一样的随机数

            loc1 = random.randint(0, num - 1)
            loc2 = random.randint(0, num - 1)
            loc3 = random.randint(0, num - 1)
            loc4 = random.randint(0, num - 1)
            loc5 = random.randint(0, num - 1)

            if len(set([loc1, loc2, loc3, loc4, loc5])) == 5:
                used = [loc1, loc2, loc3, loc4, loc5]
                # print(used)
                break

        loc = [loc1, loc2, loc3, loc4, loc5]
        dis = [0, 0, 0, 0, 0]
        # 完成第一个订单需要移动距离
        for i in range(5):
            dis[i] += ls[loc[i]][1]

        # 订单选取信息
        for i in range(5):
            order[i].append(ls[loc[i]][2])

        # 完成第一个订单员工位置
        new = [loc1, loc2, loc3, loc4, loc5]

        while len(used) < len(ls):
            mins = min(dis)
            index = dis.index(mins)

            up_min = [1000, 0, 0]

            for i in range(len(ls)):
                if i not in used:
                    if abs(ls[i][0] - new[index]) < up_min[0]:
                        up_min[0] = abs(ls[i][0] - new[index])
                        up_min[1] = i
                        up_min[2] = 0
                    if abs(ls[i][1] - new[index]) < up_min[0]:
                        up_min[0] = abs(ls[i][0] - new[index])
                        up_min[1] = i
                        up_min[2] = 1
            # 更新总距离
            dis[index] += up_min[0] + abs(ls[up_min[1]][1] - ls[up_min[1]][0])
            # 更新那单顺序
            order[index].append(ls[up_min[1]][2])
            # 更新完成订单后位置信息
            new[index] = ls[up_min[1]][1 - up_min[2]]

            used.append(up_min[1])
        for i in range(5):
            dis[i] += new[i]
        valuenew = max(dis)

        # print (valuenew)
        if valuenew < valuecurrent:  # 接受该解

            # 更新solutioncurrent 和solutionbest
            valuecurrent = valuenew
            solutioncurrent = order.copy()

            if valuenew < valuebest:
                valuebest = valuenew
                solutionbest = order.copy()
        else:  # 按必定的几率接受该解
            if np.random.rand() < np.exp(-(valuenew - valuecurrent) / t):
                valuecurrent = valuenew
                solutioncurrent = order.copy()
            else:
                order = solutioncurrent.copy()

    t = alpha * t
    result.append(valuebest)
    print('\r' + str(t), end='')  # 程序运行时间较长，打印t来监视程序进展速度

with open('result4.txt', 'a', encoding='GBK') as f:
    for i in range(5):
        f.write(str(indexs + 1) + ' ' + str(i + 1) + ' ' + ','.join(solutionbest[i]) + '\n')

0.9920974201040585

In [ ]:
with open('result59.csv', 'r') as f:
    lines = f.readlines()

data = []

for line in lines:
    line = line.strip("'\n ").split(',')
    data.append(line)
with open('result.txt', 'r') as f:
    lines = f.readlines()
d = {}
for line in lines:
    line = line.strip()
    d[line.split()[0]] = line.split()[1].split(',')

In [ ]:
for i in range(59):
    matrix = pd.read_csv('met.csv', index_col=0)

    matrix = matrix.loc[data[i]]
    matrix = matrix[d.get(str(i))]

    matrix = matrix.T

    right = matrix.apply(lambda k: len(list(k)) - list(k)[::-1].index(1)).tolist()
    left = matrix.apply(lambda k: list(k).index(1) + 1).tolist()
    pos = [[left[i],right[i],data[0][i]] for i in range(len(left))]

In [ ]:
d = {}

l1, l2, l3, l4 = [], [], [], []
ls = []
with open('result3.txt', 'r') as f:
    lines = f.readlines()
for line in lines:
    line = line.strip().split()
    line[2] = line[2].split(',')

    ls.append(line[0])
    for i in range(len(line[2])):
        l1.append(line[0])
        l2.append(line[1])
        l3.append(i + 1)
        l4.append(line[2][i])
da = pd.DataFrame({'OrderNo': l4, 'GroupNo': l1, 'WorkerNo': l2, 'TaskNo': l3})
# 字符串转为数字形式方便值大小排序
da[['GroupNo', 'WorkerNo', 'TaskNo']] = da[['GroupNo', 'WorkerNo', 'TaskNo']].astype(int)
# 排序
da = da.sort_values(by=['GroupNo', 'WorkerNo', 'TaskNo'])
da.to_csv('result3.csv', encoding='GBK', index=None)

da.head(10)